# 05. MCP Integration - Model Context Protocol 완전 이해

**MCP (Model Context Protocol)**의 아키텍처와 동작 원리를 이해하고, 실제 MCP Server를 구현합니다.

## 🎯 학습 목표

1. **MCP 아키텍처**: Host, Client, Server의 역할과 관계
2. **통신 방식**: stdin/stdout + JSON-RPC 2.0
3. **Server 구현**: 4개 외부 API를 MCP Server로 통합
4. **실전**: FastMCP로 3분 배포 + LangGraph 통합

## ⚠️ 핵심 포인트

**MCP는 도구의 "USB 표준"입니다.**

한 번 만든 Server를 모든 곳에서 재사용:
- Claude Desktop
- VSCode / Cursor
- LangGraph Agent (직접 통합!)

## Part 1: MCP 아키텍처 - Host, Client, Server 🏗️

### 1.1 전체 구조

```
┌──────────────────────────────────────────────────────┐
│  HOST (애플리케이션)                                  │
│  예: Claude Desktop, VSCode, LangGraph Agent        │
│                                                       │
│  ┌─────────┐      ┌──────────────────┐             │
│  │  LLM    │ ◀───▶│  MCP Client      │             │
│  │  (AI)   │      │  (JSON-RPC 통신) │             │
│  └─────────┘      └────────┬─────────┘             │
└───────────────────────────┼───────────────────────┘
                            │
                  stdin/stdout (JSON-RPC 2.0)
                            │
┌───────────────────────────┼───────────────────────┐
│  MCP SERVER (도구 제공자)  │                        │
│                 ┌─────────┴────────┐               │
│                 │  Server Core     │ ← MCPServer 클래스
│                 │  - tools/list    │               │
│                 │  - tools/call    │               │
│                 └──────┬───────────┘               │
│                        │                            │
│        ┌───────┬───────┼───────┬────────┐         │
│        ▼       ▼       ▼       ▼        ▼         │
│    ┌────┐  ┌────┐  ┌────┐  ┌────┐              │
│    │Tool│  │Tool│  │Tool│  │Tool│              │
│    │ 1  │  │ 2  │  │ 3  │  │ 4  │              │
│    └──┬─┘  └──┬─┘  └──┬─┘  └──┬─┘              │
└───────┼───────┼───────┼───────┼─────────────────┘
        │       │       │       │
        ▼       ▼       ▼       ▼
   Wikipedia Weather Exchange  AI
```

### 1.2 각 컴포넌트 역할

| 컴포넌트 | 누가 만드나? | 역할 | 예시 |
|---------|-------------|------|------|
| **Host** | Anthropic, MS, 개발자 | LLM 애플리케이션 | Claude Desktop, LangGraph |
| **Client** | Host에 내장 | MCP 통신 담당 | MCP SDK, langchain-mcp-adapters |
| **Server** | 개발자 (우리!) | 도구 제공 | Wikipedia Server |

### 1.3 통신: stdin/stdout + JSON-RPC

**왜 stdio?**
- 간단: 표준 입출력만 사용
- 언어 무관: Python, Node.js, Rust 모두 가능
- 격리: Server 크래시 → Host 안전

**흐름**:
```
1. Host가 Server 프로세스 시작: python server.py
2. Client → stdin → JSON 요청
3. Server → stdout → JSON 응답
4. 양방향 통신 유지 (무한 루프)
```

### 1.4 실습: 개념 확인

**시나리오**: Claude Desktop에서 Wikipedia 검색

- Host는? → Claude Desktop (Anthropic 앱)
- Client는? → Claude Desktop 내장 MCP Client
- Server는? → Wikipedia MCP Server (개발자가 만듦)

**시나리오**: LangGraph Agent에서 Wikipedia 사용

- Host는? → 내 LangGraph Agent
- Client는? → langchain-mcp-adapters
- Server는? → Wikipedia MCP Server (개발자가 만듦)

In [ ]:
# 정답 확인
print("✅ Host: 애플리케이션 (Claude Desktop, LangGraph 등)")
print("✅ Client: Host 내장 (MCP SDK, langchain-mcp-adapters)")
print("✅ Server: 개발자가 만듦 (Wikipedia, GitHub 등)")
print("\n💡 핵심: Server만 우리가 만들면 됩니다!")

In [ ]:
# Host-Client-Server 예제

print("=" * 60)
print("예제 1: Claude Desktop + Wikipedia MCP Server")
print("-" * 60)
print("Host: Claude Desktop")
print("Client: Claude Desktop 내장 MCP Client")
print("Server: Wikipedia MCP Server")
print()
print("=" * 60)
print("예제 2: LangGraph Agent + Weather MCP Server")
print("-" * 60)
print("Host: LangGraph Agent")
print("Client: langchain-mcp-adapters")
print("Server: Weather MCP Server")
print("=" * 60)

## Part 2: JSON-RPC 2.0 통신 프로토콜 📡

### 2.1 기본 형식

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/list"
}
```

### 2.2 MCP 핵심 메서드 2개

**① tools/list**: 도구 목록 조회  
**② tools/call**: 도구 실행

### 2.3 tools/list 예제

**요청**:
```json
{"jsonrpc": "2.0", "id": 1, "method": "tools/list"}
```

**응답**:
```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "tools": [
      {
        "name": "search_wikipedia",
        "description": "Wikipedia 검색",
        "inputSchema": {
          "type": "object",
          "properties": {"topic": {"type": "string"}}
        }
      }
    ]
  }
}
```

### 2.4 tools/call 예제

**요청**:
```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "method": "tools/call",
  "params": {
    "name": "search_wikipedia",
    "arguments": {"topic": "Python"}
  }
}
```

**응답**:
```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "content": [{"type": "text", "text": "Python is..."}]
  }
}
```

In [ ]:
import json

# 완성된 JSON-RPC 메시지 생성 예제

def create_list_request(req_id=1):
    return {
        "jsonrpc": "2.0",
        "id": req_id,
        "method": "tools/list"
    }

def create_call_request(tool_name, args, req_id=2):
    return {
        "jsonrpc": "2.0",
        "id": req_id,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": args
        }
    }

# 예제 1: tools/list
req1 = create_list_request(1)
print("📤 tools/list 요청:")
print(json.dumps(req1, indent=2, ensure_ascii=False))

# 예제 2: tools/call (날씨 조회)
req2 = create_call_request("get_weather", {"city": "seoul"}, 2)
print("\n📤 tools/call 요청 (날씨):")
print(json.dumps(req2, indent=2, ensure_ascii=False))

# 예제 3: tools/call (Wikipedia)
req3 = create_call_request("search_wikipedia", {"topic": "Python", "sentences": 3}, 3)
print("\n📤 tools/call 요청 (Wikipedia):")
print(json.dumps(req3, indent=2, ensure_ascii=False))

## Part 3: MCP Server 구현 🛠️

### 3.1 구현할 4개 도구

1. 🔍 Wikipedia - 검색
2. 🌤️ Weather - 날씨
3. 💱 Exchange - 환율
4. 🎨 AI Image - 이미지 생성

### 3.2 구현 전략

```
외부 API → Python 함수 (Tool) → Server Core → stdin/stdout
```

**교육 목적**:
- **이번 노트북**: 원리 이해를 위해 수동 구현
- **FastMCP**: 실무에서는 자동화 (Part 4에서 배움)

### 3.3 Tool 함수 구현


**패키지 설치**:
```bash
pip install wikipedia requests
```

**💡 중요**: 이 함수들은 단순 Python 함수입니다. MCP 통신은 나중에 추가됩니다!

In [ ]:
# 패키지 설치
!pip install wikipedia requests -q

print("✅ 패키지 설치 완료!")

In [ ]:
import wikipedia
import requests
import urllib.parse

# Tool 1: Wikipedia
def tool_wikipedia(args):
    topic = args["topic"]
    sentences = args.get("sentences", 3)
    try:
        wikipedia.set_lang("ko")
        return f"📚 {topic}:\n{wikipedia.summary(topic, sentences=sentences)}"
    except:
        return "❌ 검색 실패"

# Tool 2: Weather
def tool_weather(args):
    city = args["city"]
    coords = {"seoul": (37.5665, 126.9780), "london": (51.5074, -0.1278)}
    if city.lower() not in coords:
        return f"❌ 지원 안 함: {city}"
    lat, lon = coords[city.lower()]
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    data = requests.get(url, timeout=5).json()
    w = data["current_weather"]
    return f"🌤️ {city} 날씨:\n온도: {w['temperature']}°C, 풍속: {w['windspeed']} km/h"

# Tool 3: Exchange
def tool_exchange(args):
    from_cur, to_cur = args["from_currency"], args["to_currency"]
    url = f"https://api.exchangerate-api.com/v4/latest/{from_cur}"
    rate = requests.get(url, timeout=5).json()["rates"][to_cur]
    return f"💱 환율: 1 {from_cur} = {rate} {to_cur}"

# Tool 4: AI Image
def tool_ai_image(args):
    prompt = urllib.parse.quote(args["prompt"])
    return f"🎨 이미지 생성:\nhttps://image.pollinations.ai/prompt/{prompt}"

# Registry
TOOLS = {
    "search_wikipedia": tool_wikipedia,
    "get_weather": tool_weather,
    "get_exchange_rate": tool_exchange,
    "generate_ai_image": tool_ai_image
}

print(f"✅ {len(TOOLS)}개 도구 함수 구현 완료")
print("\n⚠️ 주의: 아직 MCP Server가 아닙니다! 단순 Python 함수입니다.")

### 3.3.1 Q: 위 코드를 FastMCP에 넣으면 자동으로 TOOL_DEF 생성되나요?

**A: 안 됩니다!** ❌

위 코드는 **타입 힌트가 없어서** FastMCP가 TOOL_DEF를 자동 생성할 수 없습니다.

---

**Q: FastMCP는 어떻게 자동 생성하나요? LLM이 달렸나요?**

**A: 아니요! Python 코드 분석입니다!** 🐍

FastMCP는 Python의 **타입 힌트 + docstring**을 파싱해서 JSON Schema를 자동 생성합니다.

**원리** (FastMCP 내부 동작):

```python
import inspect
from typing import get_type_hints

# 1. 함수 시그니처 분석
def search_wikipedia(topic: str, sentences: int = 3) -> str:
    '''Wikipedia에서 주제를 검색합니다'''
    pass

# 2. FastMCP가 내부적으로 하는 일
sig = inspect.signature(search_wikipedia)
hints = get_type_hints(search_wikipedia)
doc = search_wikipedia.__doc__

# 3. TOOL_DEF 생성
tool_def = {
    "name": "search_wikipedia",
    "description": doc.split('
')[0],  # ← docstring 첫 줄
    "inputSchema": {
        "type": "object",
        "properties": {},
        "required": []
    }
}

# 4. 각 파라미터 분석
for param_name, param in sig.parameters.items():
    if param_name == 'return':
        continue
    
    # 타입 힌트 → JSON Schema 타입
    python_type = hints.get(param_name, str)
    json_type = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean"
    }.get(python_type, "string")
    
    tool_def["inputSchema"]["properties"][param_name] = {
        "type": json_type
    }
    
    # 기본값이 없으면 required
    if param.default == inspect.Parameter.empty:
        tool_def["inputSchema"]["required"].append(param_name)
    else:
        tool_def["inputSchema"]["properties"][param_name]["default"] = param.default
```

**결과**:
```json
{
  "name": "search_wikipedia",
  "description": "Wikipedia에서 주제를 검색합니다",
  "inputSchema": {
    "type": "object",
    "properties": {
      "topic": {"type": "string"},
      "sentences": {"type": "integer", "default": 3}
    },
    "required": ["topic"]
  }
}
```

---

**핵심**:
- ❌ LLM이 아님!
- ✅ Python `inspect` + `typing` 모듈로 코드 분석
- ✅ 타입 힌트 → JSON Schema 자동 변환
- ✅ docstring → description

**그래서** Part 3에서는 수동으로 TOOL_DEF를 작성했고,  
Part 4에서 FastMCP + 타입 힌트로 자동화합니다!

### 3.4 TOOL_DEFS - LLM에게 사용법 알려주기

**🤔 문제 상황**:

위에서 Tool 함수 4개를 만들었습니다:
- `tool_wikipedia(args)` ✅
- `tool_weather(args)` ✅
- `tool_exchange(args)` ✅
- `tool_ai_image(args)` ✅

**하지만!** LLM은 이걸 어떻게 사용하는지 모릅니다.

```python
# LLM 입장:
def tool_wikipedia(args):  # ← args에 뭐가 들어가야 하지?
    topic = args["topic"]   # ← "topic"이라는 키가 있다는 걸 어떻게 알지?
    sentences = args.get("sentences", 3)  # ← 선택 항목인지 필수인지?
```

LLM이 알아야 하는 것:
- 함수 이름이 뭔지 ("search_wikipedia")
- 어떤 인자가 필요한지 (topic, sentences)
- 각 인자가 필수인지 선택인지
- 각 인자의 타입은 뭔지 (string, integer)
- 기본값은 뭔지 (sentences=3)

**해결책: TOOL_DEFS (도구 명세서)!**

```
LLM: "Wikipedia 검색하고 싶은데 어떻게 쓰지?"
Server: "여기 TOOL_DEFS 봐!"
        {
          "name": "search_wikipedia",
          "inputSchema": {
            "properties": {
              "topic": {"type": "string"},  ← 필수
              "sentences": {"type": "integer", "default": 3}  ← 선택
            }
          }
        }
LLM: "아하! 그럼 {"topic": "Python", "sentences": 3} 보낼게"
```

**Q: 왜 수동으로 작성하나요?**

**교육 목적**: FastMCP가 뒤에서 자동으로 생성하는 것을 직접 보기 위함

| 방식 | TOOL_DEFS | 장점 |
|------|-----------|------|
| **수동 (이번)** | 직접 작성 | 원리 이해 |
| **FastMCP** | 타입 힌트에서 자동 생성 | 빠르고 편함 |

**FastMCP 자동 생성 예**:
```python
@mcp.tool()
def search_wikipedia(topic: str, sentences: int = 3) -> str:
    # ↑ 타입 힌트에서 inputSchema 자동 생성!
    pass
```

In [ ]:
# 수동 작성 (교육용)
TOOL_DEFS = {
    "search_wikipedia": {
        "name": "search_wikipedia",
        "description": "Wikipedia 검색",
        "inputSchema": {
            "type": "object",
            "properties": {
                "topic": {"type": "string"},
                "sentences": {"type": "integer", "default": 3}
            },
            "required": ["topic"]
        }
    },
    "get_weather": {
        "name": "get_weather",
        "description": "날씨 조회",
        "inputSchema": {
            "type": "object",
            "properties": {"city": {"type": "string"}},
            "required": ["city"]
        }
    },
    "get_exchange_rate": {
        "name": "get_exchange_rate",
        "description": "환율 조회",
        "inputSchema": {
            "type": "object",
            "properties": {
                "from_currency": {"type": "string"},
                "to_currency": {"type": "string"}
            },
            "required": ["from_currency", "to_currency"]
        }
    },
    "generate_ai_image": {
        "name": "generate_ai_image",
        "description": "AI 이미지 생성",
        "inputSchema": {
            "type": "object",
            "properties": {"prompt": {"type": "string"}},
            "required": ["prompt"]
        }
    }
}

print(f"✅ {len(TOOL_DEFS)}개 정의 완료")
print("\n💡 FastMCP는 이걸 자동으로 생성합니다!")

### 3.5 MCP Server Core

**이것이 아키텍처 다이어그램의 "Server Core"입니다!**

JSON-RPC 요청을 받아서 처리하는 핵심 로직:
- `tools/list` → TOOL_DEFS 반환
- `tools/call` → Tool 함수 실행

**⚠️ 주의**: 아직 stdin/stdout 통신은 없습니다 (3.7에서 추가)

In [ ]:
class MCPServer:
    """MCP Server Core - JSON-RPC 요청 처리"""
    
    def __init__(self, tool_defs, tool_funcs):
        self.defs = tool_defs
        self.funcs = tool_funcs
    
    def handle(self, request):
        """JSON-RPC 요청 처리"""
        method = request.get("method")
        req_id = request.get("id")
        
        if method == "tools/list":
            return {
                "jsonrpc": "2.0",
                "id": req_id,
                "result": {"tools": list(self.defs.values())}
            }
        
        elif method == "tools/call":
            tool_name = request["params"]["name"]
            args = request["params"]["arguments"]
            
            if tool_name not in self.funcs:
                text = f"❌ 도구 없음: {tool_name}"
            else:
                try:
                    text = self.funcs[tool_name](args)
                except Exception as e:
                    text = f"❌ 오류: {str(e)}"
            
            return {
                "jsonrpc": "2.0",
                "id": req_id,
                "result": {"content": [{"type": "text", "text": text}]}
            }
        
        return {
            "jsonrpc": "2.0",
            "id": req_id,
            "error": {"message": f"Unknown method: {method}"}
        }

server = MCPServer(TOOL_DEFS, TOOLS)
print("✅ MCP Server Core 생성 완료!")
print("\n⚠️ 주의: 아직 통신 레이어가 없습니다!")

### 3.6 테스트 (노트북에서만 가능)

노트북에서는 직접 함수 호출로 테스트합니다.

**실제 MCP Server는 stdin/stdout으로 통신합니다 (3.7 참고)!**

In [ ]:
# Test 1: tools/list
req1 = create_list_request(1)
res1 = server.handle(req1)
print("🧪 Test 1: tools/list")
print(f"등록 도구: {len(res1['result']['tools'])}개\n")

# Test 2: Wikipedia
req2 = create_call_request("search_wikipedia", {"topic": "Python", "sentences": 2}, 2)
res2 = server.handle(req2)
print("🧪 Test 2: Wikipedia")
print(res2['result']['content'][0]['text'][:100] + "...\n")

# Test 3: Weather
req3 = create_call_request("get_weather", {"city": "seoul"}, 3)
res3 = server.handle(req3)
print("🧪 Test 3: Weather")
print(res3['result']['content'][0]['text'] + "\n")

# Test 4: Exchange
req4 = create_call_request("get_exchange_rate", {"from_currency": "USD", "to_currency": "KRW"}, 4)
res4 = server.handle(req4)
print("🧪 Test 4: Exchange")
print(res4['result']['content'][0]['text'] + "\n")

# Test 5: AI Image
req5 = create_call_request("generate_ai_image", {"prompt": "sunset"}, 5)
res5 = server.handle(req5)
print("🧪 Test 5: AI Image")
print(res5['result']['content'][0]['text'])

# Test 6: 에러 케이스
print("\n🧪 Test 6: 에러 케이스")
req_err = create_call_request("invalid_tool", {}, 6)
res_err = server.handle(req_err)
print(res_err['result']['content'][0]['text'])

print("\n✅ 모든 테스트 완료!")

### 3.7 실제 MCP Server로 만들려면?

**노트북에서는 여기까지만 가능합니다!**

**왜?**
- 실제 MCP Server는 **무한 루프**로 stdin/stdout 통신을 해야 함
- 노트북에서 무한 루프를 돌리면 → 셀이 계속 실행 중 → 다음 개발 불가능

```python
# 이런 코드가 필요한데...
while True:  # ← 무한 루프
    request = sys.stdin.readline()  # Client 대기
    response = server.handle(request)
    print(response, flush=True)
```

**해결책**: 별도 파일로 저장하고 터미널에서 실행!

하지만 이것도 귀찮습니다... **FastMCP가 이걸 다 자동으로 해줍니다!**

## Part 4: FastMCP로 3분 배포 🚀

### 4.1 비교

| 항목 | 이번 노트북 (Part 3) | FastMCP |
|------|---------------------|---------|
| 목적 | 원리 이해 | 빠른 구축 |
| 코드 | ~150줄 | ~10줄 |
| JSON-RPC | 수동 작성 | 자동 생성 |
| stdin/stdout | 수동 구현 | 자동 처리 |
| TOOL_DEFS | 수동 작성 | 타입 힌트에서 자동 |

**FastMCP가 자동으로 해주는 것들**:
- ✅ TOOL_DEFS 생성 (타입 힌트에서 자동)
- ✅ Server Core 구현 (JSON-RPC 처리)
- ✅ stdin/stdout 통신 (무한 루프)
- ✅ 에러 핸들링

### 4.2 FastMCP로 실제 MCP Server 만들기

**이제부터가 실전입니다!**

**설치**:
```bash
pip install fastmcp wikipedia
```

**server.py 파일 생성** (이것만 저장하면 됩니다!):

```python
from fastmcp import FastMCP
import wikipedia

mcp = FastMCP("Wikipedia Server")

@mcp.tool()
def search_wikipedia(topic: str, sentences: int = 3) -> str:
    '''Wikipedia에서 주제를 검색합니다'''
    wikipedia.set_lang("ko")
    return wikipedia.summary(topic, sentences=sentences)

if __name__ == "__main__":
    mcp.run()
```

**끝!** 이게 전부입니다.

**FastMCP가 자동으로 해주는 것들**:
- ✅ TOOL_DEFS 생성 (타입 힌트에서 자동)
- ✅ Server Core 구현 (JSON-RPC 처리)
- ✅ stdin/stdout 통신 (무한 루프)
- ✅ 에러 핸들링

**비교**:
- 수동 구현: ~150줄 (교육용, 저장 불필요)
- **FastMCP: ~10줄 (실무용, 실제 사용!)** ✅

**저장 위치**:
```bash
$ cd /path/to/project
$ touch server.py
# 위 코드 복사해서 저장
```

### 4.3 FastMCP 실행 방법

**1. server.py 작성** (위 코드)

**2. 터미널에서 실행**:
```bash
$ python server.py
# 또는
$ fastmcp run server.py
```

**3. 무슨 일이 일어나나?**
```
1. FastMCP가 stdin에서 대기
2. Client 연결 (Claude Desktop, LangGraph 등)
3. JSON-RPC 메시지 주고받기
4. Client 종료 시 Server도 종료
```

**Claude Desktop 설정** (~/.claude/config.json):
```json
{
  "mcpServers": {
    "wikipedia": {
      "command": "python",
      "args": ["path/to/server.py"]
    }
  }
}
```

**LangGraph에서 사용**:
```python
client = MultiServerMCPClient({
    "wikipedia": {
        "command": "python",
        "args": ["path/to/server.py"]
    }
})
```

**💡 핵심**: 한 번만 만들면 어디서든 재사용!

## Part 5: LangGraph에서 MCP Server 사용하기 🔗

**핵심 질문**: LangGraph Agent가 MCP Server를 어떻게 호출하나?

**답**: `langchain-mcp-adapters` 패키지 사용!

### 5.1 설치

```bash
pip install langchain-mcp-adapters langgraph langchain-openai
```

### 5.2 LangGraph + MCP 통합 (create_react_agent 방식)

**전체 흐름**:
```
LangGraph Agent → MCP Client → server.py (stdin/stdout) → 외부 API
```

**방법 1: create_react_agent 사용 (간편)**

```python
import asyncio
from langchain_mcp_adapters import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

async def run_agent():
    # 1. MCP Client 설정
    client = MultiServerMCPClient({
        "wikipedia": {
            "command": "python",
            "args": ["server.py"],  # ← FastMCP로 만든 Server
            "transport": "stdio"
        }
    })
    
    # 2. MCP Tools 가져오기
    async with client:
        tools = await client.get_tools()
        
        # 3. Agent 생성
        llm = ChatOpenAI(model="gpt-4o-mini")
        agent = create_react_agent(llm, tools)
        
        # 4. 실행
        result = await agent.ainvoke({
            "messages": [("user", "Python 프로그래밍 언어에 대해 알려줘")]
        })
        
        print(result["messages"][-1].content)

# 실행
# asyncio.run(run_agent())
```

**동작 과정**:
1. Agent가 "Wikipedia 검색 필요"라고 판단
2. MCP Client가 `python server.py` 실행
3. stdin으로 JSON 전송: `{"method": "tools/call", ...}`
4. server.py가 Wikipedia API 호출
5. stdout으로 결과 반환
6. Agent가 최종 답변 생성

### 5.3 LangGraph + MCP 통합 (StateGraph 방식)

**방법 2: StateGraph 직접 구성 (세밀한 제어)**

학생들이 배운 StateGraph 방식으로도 MCP를 통합할 수 있습니다!

```python
import asyncio
from typing import TypedDict, Annotated
from langchain_mcp_adapters import MultiServerMCPClient
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode

# 1. State 정의
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 전체를 async 함수로 (MCP Client 생명주기 유지!)
async def main():
    # MCP Client: async with로 연결 유지
    async with MultiServerMCPClient({
        "wikipedia": {
            "command": "python",
            "args": ["server.py"],
            "transport": "stdio"
        }
    }) as client:
        # Tools 가져오기
        tools = await client.get_tools()
        
        # 3. Agent Node
        def agent_node(state: AgentState):
            llm = ChatOpenAI(model="gpt-4o-mini")
            llm_with_tools = llm.bind_tools(tools)
            response = llm_with_tools.invoke(state["messages"])
            return {"messages": [response]}
        
        # 4. Should Continue
        def should_continue(state: AgentState):
            last_message = state["messages"][-1]
            if last_message.tool_calls:
                return "tools"
            return END
        
        # 5. Graph 구성
        graph = StateGraph(AgentState)
        graph.add_node("agent", agent_node)
        graph.add_node("tools", ToolNode(tools))
        
        graph.add_edge(START, "agent")
        graph.add_conditional_edges("agent", should_continue, ["tools", END])
        graph.add_edge("tools", "agent")
        
        app = graph.compile()
        
        # 6. 실행 (async with 블록 안에서!)
        result = await app.ainvoke({
            "messages": [("user", "Python이 뭐야?")]
        })
        print(result["messages"][-1].content)

# 실행
asyncio.run(main())
```

**💡 핵심**: `async with` 블록 안에서 Graph를 구성하고 실행해야 MCP Server 연결이 유지됩니다!

---

**StateGraph vs create_react_agent 비교**:

| 항목 | create_react_agent | StateGraph |
|------|-------------------|-----------|
| 코드량 | 짧음 (~15줄) | 길음 (~50줄) |
| 제어 | 자동 (ReAct 패턴) | 수동 (세밀한 제어) |
| 커스터마이징 | 어려움 | 쉬움 |
| 학습 곡선 | 낮음 | 높음 |
| MCP Client 관리 | 자동 | 수동 (async with 필요) |
| 추천 | 빠른 프로토타입 | 복잡한 워크플로우 |

**언제 StateGraph를 쓰나?**
- ✅ Human-in-the-Loop 필요
- ✅ 다중 Agent 협업
- ✅ 복잡한 조건 분기
- ✅ 커스텀 State 관리

**언제 create_react_agent를 쓰나?**
- ✅ 빠른 구현
- ✅ 단순 ReAct 패턴
- ✅ 기본 도구 사용만 필요

## Part 6: 결론 ⚖️

### 장점 ✅
- **재사용성**: 한 번만 만들면 모든 Host에서 사용
- **표준화**: JSON-RPC 2.0
- **N:M 매칭**: 여러 Host ↔ 여러 Server
- **LangGraph 통합**: langchain-mcp-adapters로 쉽게 연결

### 단점 ❌
- **복잡도**: 간단한 API도 래핑 필요
- **오버헤드**: JSON 직렬화 + 프로세스 간 통신

### 언제 사용?

**✅ 권장**: 도구 3-15개, 재사용 필요, 팀 협업, LangGraph 통합  
**❌ 불필요**: 도구 1-2개, 빠른 프로토타입

### 추천 사항

1. **학습**: 이번 노트북 (원리 이해)
2. **실무**: FastMCP (3분 구축)
3. **통합**: langchain-mcp-adapters (LangGraph 연결)

---

## 🎓 완료!

**핵심 정리**:

1. **Host-Client-Server** 아키텍처 이해
2. **JSON-RPC 2.0** 통신 프로토콜
3. **Server Core + stdin/stdout** 구현
4. **FastMCP** 자동화
5. **LangGraph 통합** 방법

**실전 활용**:
```python
# 1. FastMCP로 Server 만들기
from fastmcp import FastMCP
mcp = FastMCP("My Tools")

@mcp.tool()
def my_tool(x: str) -> str:
    return f"Result: {x}"

# 2. LangGraph에서 사용
from langchain_mcp_adapters import MultiServerMCPClient
client = MultiServerMCPClient({"my": {"command": "python", "args": ["server.py"]}})
tools = await client.get_tools()

# 3. Agent 생성
agent = create_react_agent(llm, tools)
```

**다음 단계**: 06-advanced-rag-agent.ipynb